In [2]:
# ==========================================
# Global model (LGBM) + Baselines (NO FILE OUTPUT)
# Target week: ###
# - Print key tables to cell output
# - DO NOT save any CSV files
# ==========================================

import os
import numpy as np
import pandas as pd
import lightgbm as lgb

# --------------------------
# Config
# --------------------------
DATA_PATH = "../../final_data/data_260119_random.csv"
TARGET_WEEK = "2025-12-29"

CAT_COLS = ["treatment", "source", "ops_type_merged", "city_group"]
DATE_COL = "experiment_date"

Y1 = "trip_cnt_per_user"
Y2 = "nonrepeat_cnt_per_user"

Y1_LAG1 = "trip_cnt_per_user_lag1"
Y1_ROLL4 = "trip_cnt_per_user_roll4"
Y2_LAG1 = "nonrepeat_cnt_per_user_lag1"
Y2_ROLL4 = "nonrepeat_cnt_per_user_roll4"

DROP_FEATURES = {"log1p_user_cnt", "week_idx"}
RANDOM_STATE = 42

# Print options (avoid truncation in console as much as possible)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# --------------------------
# Helpers
# --------------------------
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def mae(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.mean(np.abs(y_true - y_pred)))

def make_time_val_split(train_df, date_col, n_val_weeks=4):
    uniq_dates = np.array(sorted(pd.to_datetime(train_df[date_col]).dt.date.unique()))
    if len(uniq_dates) <= 2:
        return train_df.copy(), None

    n_val_weeks = min(n_val_weeks, max(1, len(uniq_dates) // 5))
    val_dates = set(uniq_dates[-n_val_weeks:])

    tr = train_df[~pd.to_datetime(train_df[date_col]).dt.date.isin(val_dates)].copy()
    va = train_df[pd.to_datetime(train_df[date_col]).dt.date.isin(val_dates)].copy()

    if tr.empty or va.empty:
        cut = int(len(uniq_dates) * 0.8)
        cut = max(1, min(cut, len(uniq_dates) - 1))
        tr_dates = set(uniq_dates[:cut])
        va_dates = set(uniq_dates[cut:])
        tr = train_df[pd.to_datetime(train_df[date_col]).dt.date.isin(tr_dates)].copy()
        va = train_df[pd.to_datetime(train_df[date_col]).dt.date.isin(va_dates)].copy()
        if tr.empty or va.empty:
            return train_df.copy(), None

    return tr, va

def metrics_by_group(test_df, group_col, y_col, pred_cols):
    rows = []
    for key, g in test_df.groupby(group_col):
        y = g[y_col].astype(float).values
        n = len(g)
        for model_name, pred_col in pred_cols.items():
            rows.append({
                group_col: key,
                "target": y_col,
                "model": model_name,
                "MAE": mae(y, g[pred_col].astype(float).values),
                "RMSE": rmse(y, g[pred_col].astype(float).values),
                "n": n,
            })
    return pd.DataFrame(rows).sort_values(["target", group_col, "model"]).reset_index(drop=True)

def group_rmse_table(test_df, y_col, pred_col, group_cols):
    rows = []
    for key, g in test_df.groupby(group_cols):
        y = g[y_col].astype(float).values
        p = g[pred_col].astype(float).values
        rows.append({
            **{col: val for col, val in zip(group_cols, key)},
            "target": y_col,
            "RMSE": rmse(y, p),
            "MAE": mae(y, p),
            "n": len(g),
        })
    return pd.DataFrame(rows).sort_values(["target", "RMSE"]).reset_index(drop=True)

def print_df(df_, title, head_n=None):
    print(f"\n=== {title} ===")
    if head_n is None:
        print(df_.to_string(index=False))
    else:
        print(df_.head(head_n).to_string(index=False))
        if len(df_) > head_n:
            print(f"... (printed first {head_n} rows; total rows={len(df_)})")

# --------------------------
# Load & preprocess
# --------------------------
df = pd.read_csv(DATA_PATH)
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")

if df[DATE_COL].isna().any():
    bad = df[df[DATE_COL].isna()].head(10)
    raise ValueError(f"Found unparsable dates in {DATE_COL}. Examples:\n{bad}")

required_cols = CAT_COLS + [DATE_COL, Y1, Y2, Y1_LAG1, Y1_ROLL4, Y2_LAG1, Y2_ROLL4]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

for c in CAT_COLS:
    df[c] = df[c].astype("category")

min_date = df[DATE_COL].min()
df["week_idx"] = ((df[DATE_COL] - min_date).dt.days // 7).astype(int)

target_week = pd.to_datetime(TARGET_WEEK)

train = df[df[DATE_COL] < target_week].copy()
test = df[df[DATE_COL] == target_week].copy()

if train.empty:
    raise ValueError("Train set is empty. TARGET_WEEK may be too early.")
if test.empty:
    raise ValueError(f"Test set is empty for week {TARGET_WEEK}. Check experiment_date values.")

print("=== Target week ===", TARGET_WEEK)
print("Train rows:", len(train), "| Test rows:", len(test))

# --------------------------
# Baselines
# --------------------------
test["pred_y1_lag1"] = test[Y1_LAG1].astype(float)
test["pred_y1_roll4"] = test[Y1_ROLL4].astype(float)
test["pred_y2_lag1"] = test[Y2_LAG1].astype(float)
test["pred_y2_roll4"] = test[Y2_ROLL4].astype(float)

# --------------------------
# Features
# --------------------------
exclude = {Y1, Y2} | set(DROP_FEATURES)
feature_cols = [c for c in df.columns if c not in exclude]
if DATE_COL in feature_cols:
    feature_cols.remove(DATE_COL)

cat_features = [c for c in CAT_COLS if c in feature_cols]

X_train = train[feature_cols].copy()
X_test = test[feature_cols].copy()

y1_train = train[Y1].astype(float).values
y2_train = train[Y2].astype(float).values
y1_true = test[Y1].astype(float).values
y2_true = test[Y2].astype(float).values

# --------------------------
# Val split for early stopping
# --------------------------
tr_df, va_df = make_time_val_split(train, DATE_COL, n_val_weeks=4)
X_tr = tr_df[feature_cols].copy()
y1_tr = tr_df[Y1].astype(float).values
y2_tr = tr_df[Y2].astype(float).values

X_va = y1_va = y2_va = None
if va_df is not None:
    X_va = va_df[feature_cols].copy()
    y1_va = va_df[Y1].astype(float).values
    y2_va = va_df[Y2].astype(float).values

fit_kwargs = dict(categorical_feature=cat_features)

# --------------------------
# Two targets, two param sets
# --------------------------
base_params = dict(
    objective="regression",
    n_estimators=8000,
    learning_rate=0.03,
    random_state=RANDOM_STATE,
    force_row_wise=True,
)

# Y1: param 3 (min_child_samples=300)
params_y1 = dict(
    num_leaves=31, max_depth=6, min_child_samples=300,
    subsample=0.8, colsample_bytree=0.8, bagging_freq=1,
    reg_alpha=0.0, reg_lambda=2.0, min_split_gain=0.0,
)

# Y2: param 12 (colsample_bytree=0.9, min_child_samples=200)
params_y2 = dict(
    num_leaves=31, max_depth=6, min_child_samples=200,
    subsample=0.8, colsample_bytree=0.9, bagging_freq=1,
    reg_alpha=0.0, reg_lambda=2.0, min_split_gain=0.0,
)

m_y1 = lgb.LGBMRegressor(**{**base_params, **params_y1})
m_y2 = lgb.LGBMRegressor(**{**base_params, **params_y2})

if X_va is not None:
    m_y1.fit(
        X_tr, y1_tr,
        eval_set=[(X_va, y1_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)],
        **fit_kwargs
    )
    m_y2.fit(
        X_tr, y2_tr,
        eval_set=[(X_va, y2_va)],
        eval_metric="l2",
        callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)],
        **fit_kwargs
    )
else:
    m_y1.fit(X_train, y1_train, **fit_kwargs)
    m_y2.fit(X_train, y2_train, **fit_kwargs)

test["pred_y1_lgbm"] = m_y1.predict(X_test)
test["pred_y2_lgbm"] = m_y2.predict(X_test)

# --------------------------
# Overall metrics
# --------------------------
metrics_df = pd.DataFrame([
    {"target": Y1, "model": "baseline_lag1",  "MAE": mae(y1_true, test["pred_y1_lag1"]),  "RMSE": rmse(y1_true, test["pred_y1_lag1"])},
    {"target": Y1, "model": "baseline_roll4", "MAE": mae(y1_true, test["pred_y1_roll4"]), "RMSE": rmse(y1_true, test["pred_y1_roll4"])},
    {"target": Y1, "model": "lgbm_global",    "MAE": mae(y1_true, test["pred_y1_lgbm"]),  "RMSE": rmse(y1_true, test["pred_y1_lgbm"])},
    {"target": Y2, "model": "baseline_lag1",  "MAE": mae(y2_true, test["pred_y2_lag1"]),  "RMSE": rmse(y2_true, test["pred_y2_lag1"])},
    {"target": Y2, "model": "baseline_roll4", "MAE": mae(y2_true, test["pred_y2_roll4"]), "RMSE": rmse(y2_true, test["pred_y2_roll4"])},
    {"target": Y2, "model": "lgbm_global",    "MAE": mae(y2_true, test["pred_y2_lgbm"]),  "RMSE": rmse(y2_true, test["pred_y2_lgbm"])},
]).sort_values(["target", "model"]).reset_index(drop=True)

# --------------------------
# Metrics by ops_type_merged / city_group / treatment
# --------------------------
pred_map_y1 = {"baseline_lag1": "pred_y1_lag1", "baseline_roll4": "pred_y1_roll4", "lgbm_global": "pred_y1_lgbm"}
pred_map_y2 = {"baseline_lag1": "pred_y2_lag1", "baseline_roll4": "pred_y2_roll4", "lgbm_global": "pred_y2_lgbm"}

metrics_by_ops_df = pd.concat([
    metrics_by_group(test, "ops_type_merged", Y1, pred_map_y1),
    metrics_by_group(test, "ops_type_merged", Y2, pred_map_y2),
], ignore_index=True).sort_values(["target", "ops_type_merged", "model"]).reset_index(drop=True)

metrics_by_city_df = pd.concat([
    metrics_by_group(test, "city_group", Y1, pred_map_y1),
    metrics_by_group(test, "city_group", Y2, pred_map_y2),
], ignore_index=True).sort_values(["target", "city_group", "model"]).reset_index(drop=True)

metrics_by_treat_df = pd.concat([
    metrics_by_group(test, "treatment", Y1, pred_map_y1),
    metrics_by_group(test, "treatment", Y2, pred_map_y2),
], ignore_index=True).sort_values(["target", "treatment", "model"]).reset_index(drop=True)

# --------------------------
# FULL prediction output (print preview only)
# --------------------------
id_cols = [DATE_COL, "treatment", "source", "ops_type_merged", "city_group"]
pred_cols = [
    Y1, "pred_y1_lag1", "pred_y1_roll4", "pred_y1_lgbm",
    Y2, "pred_y2_lag1", "pred_y2_roll4", "pred_y2_lgbm",
]
full_pred_df = test[id_cols + pred_cols].copy()

# --------------------------
# Aggregates
# --------------------------
agg = pd.DataFrame({
    "metric": ["mean_true", "mean_pred_lag1", "mean_pred_roll4", "mean_pred_lgbm"],
    "trip_cnt_per_user": [
        float(np.mean(test[Y1])),
        float(np.mean(test["pred_y1_lag1"])),
        float(np.mean(test["pred_y1_roll4"])),
        float(np.mean(test["pred_y1_lgbm"])),
    ],
    "nonrepeat_cnt_per_user": [
        float(np.mean(test[Y2])),
        float(np.mean(test["pred_y2_lag1"])),
        float(np.mean(test["pred_y2_roll4"])),
        float(np.mean(test["pred_y2_lgbm"])),
    ],
})

# --------------------------
# Best/Worst 15 group RMSE (LGBM only)
# --------------------------
GROUP_KEY_COLS = ["treatment", "source", "ops_type_merged", "city_group"]

group_rmse_y1 = group_rmse_table(test, Y1, "pred_y1_lgbm", GROUP_KEY_COLS)
group_rmse_y2 = group_rmse_table(test, Y2, "pred_y2_lgbm", GROUP_KEY_COLS)

best15_y1 = group_rmse_y1.nsmallest(15, "RMSE").copy()
worst15_y1 = group_rmse_y1.nlargest(15, "RMSE").copy()
best15_y2 = group_rmse_y2.nsmallest(15, "RMSE").copy()
worst15_y2 = group_rmse_y2.nlargest(15, "RMSE").copy()

params_log = pd.DataFrame([
    {"target": Y1, "params": str({**base_params, **params_y1})},
    {"target": Y2, "params": str({**base_params, **params_y2})},
])

# --------------------------
# Print outputs (NO SAVING)
# --------------------------
print_df(metrics_df, "Overall metrics (target week)", head_n=None)

print_df(metrics_by_ops_df, "Metrics by ops_type_merged (target week)", head_n=80)
print_df(metrics_by_city_df, "Metrics by city_group (target week)", head_n=80)
print_df(metrics_by_treat_df, "Metrics by treatment (target week)", head_n=80)

print_df(full_pred_df, "Predictions FULL (preview first 30 rows; not saved)", head_n=30)

print_df(agg, "Quick aggregates (means)", head_n=None)

print_df(group_rmse_y1, "Group RMSE (LGBM) - trip_cnt_per_user (all groups) [preview 30]", head_n=30)
print_df(group_rmse_y2, "Group RMSE (LGBM) - nonrepeat_cnt_per_user (all groups) [preview 30]", head_n=30)

print_df(best15_y1, "Best 15 group RMSE (LGBM) - trip_cnt_per_user", head_n=None)
print_df(worst15_y1, "Worst 15 group RMSE (LGBM) - trip_cnt_per_user", head_n=None)
print_df(best15_y2, "Best 15 group RMSE (LGBM) - nonrepeat_cnt_per_user", head_n=None)
print_df(worst15_y2, "Worst 15 group RMSE (LGBM) - nonrepeat_cnt_per_user", head_n=None)

print_df(params_log, "Final model params", head_n=None)

print("\n✅ Done. Printed results only (no files were saved).")


=== Target week === 2025-12-29
Train rows: 1920 | Test rows: 192
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Total Bins 11852
[LightGBM] [Info] Number of data points in the train set: 1536, number of used features: 73
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Start training from score 0.108608
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\806566470.py:76: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for key, g in test_df.groupby(group_col):
C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\806566470.py:76: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for key, g in test_df.groupby(group_col):
C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\806566470.py:76: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default


=== Overall metrics (target week) ===
                target          model      MAE     RMSE
nonrepeat_cnt_per_user  baseline_lag1 0.053499 0.081702
nonrepeat_cnt_per_user baseline_roll4 0.044280 0.065870
nonrepeat_cnt_per_user    lgbm_global 0.041134 0.059561
     trip_cnt_per_user  baseline_lag1 0.043517 0.064229
     trip_cnt_per_user baseline_roll4 0.033838 0.051670
     trip_cnt_per_user    lgbm_global 0.030381 0.044045

=== Metrics by ops_type_merged (target week) ===
ops_type_merged                 target          model      MAE     RMSE  n
   14天在其他尖峰預估車資 nonrepeat_cnt_per_user  baseline_lag1 0.055420 0.065922 24
   14天在其他尖峰預估車資 nonrepeat_cnt_per_user baseline_roll4 0.049005 0.054897 24
   14天在其他尖峰預估車資 nonrepeat_cnt_per_user    lgbm_global 0.039380 0.046546 24
    14天在晚尖峰預估車資 nonrepeat_cnt_per_user  baseline_lag1 0.117376 0.164202 24
    14天在晚尖峰預估車資 nonrepeat_cnt_per_user baseline_roll4 0.094784 0.127268 24
    14天在晚尖峰預估車資 nonrepeat_cnt_per_user    lgbm_global 0.084198 0.1074

In [3]:
# ==========================
# Print only selected treatments (ALL rows)
# ==========================
FOCUS_TREATMENTS = ["15元1張", "20元1張", "30元1張"]

focus_df = full_pred_df[full_pred_df["treatment"].astype(str).isin(FOCUS_TREATMENTS)].copy()

print("\n==========================")
print("FOCUS treatments:", FOCUS_TREATMENTS)
print("Rows:", len(focus_df))
print("==========================\n")

if focus_df.empty:
    print("⚠️ No rows found for these treatments. Check spelling / spaces / full-width characters.")
else:
    # 排序讓你更好讀：先 treatment，再 source/ops/city，再日期
    focus_df = focus_df.sort_values(
        ["treatment", "source", "ops_type_merged", "city_group", DATE_COL]
    )

    # 全列印（不只 head）
    pd.set_option("display.max_rows", 1000000)  # 確保真的印得出全部（若太多會很長）
    pd.set_option("display.max_columns", 200)
    pd.set_option("display.width", 220)

    print("=== Row-level results (truth + lag1 + roll4 + lgbm) ===")
    print(focus_df.to_string(index=False))

    # --------------------------
    # Group-level summary (mean of truth/preds)
    # --------------------------
    grp_cols = ["treatment", "source", "ops_type_merged", "city_group"]

    group_summary = (
        focus_df
        .groupby(grp_cols, dropna=False)
        .agg(
            n=("treatment", "size"),

            y1_true_mean=(Y1, "mean"),
            y1_lag1_mean=("pred_y1_lag1", "mean"),
            y1_roll4_mean=("pred_y1_roll4", "mean"),
            y1_lgbm_mean=("pred_y1_lgbm", "mean"),

            y2_true_mean=(Y2, "mean"),
            y2_lag1_mean=("pred_y2_lag1", "mean"),
            y2_roll4_mean=("pred_y2_roll4", "mean"),
            y2_lgbm_mean=("pred_y2_lgbm", "mean"),
        )
        .reset_index()
        .sort_values(["treatment", "source", "ops_type_merged", "city_group"])
    )

    print("\n=== Group-level summary (means by treatment/source/ops/city_group) ===")
    print(group_summary.to_string(index=False))

    # （可選）每組的 RMSE / MAE（因為你要「組別結果」，很多時候也會想一起看誤差）
    def _rmse(a, b):
        a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
        return float(np.sqrt(np.mean((a - b) ** 2)))

    def _mae(a, b):
        a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
        return float(np.mean(np.abs(a - b)))

    rows = []
    for key, g in focus_df.groupby(grp_cols):
        rows.append({
            **{col: val for col, val in zip(grp_cols, key)},
            "n": len(g),

            "Y1_RMSE_lag1": _rmse(g[Y1], g["pred_y1_lag1"]),
            "Y1_RMSE_roll4": _rmse(g[Y1], g["pred_y1_roll4"]),
            "Y1_RMSE_lgbm": _rmse(g[Y1], g["pred_y1_lgbm"]),
            "Y1_MAE_lgbm": _mae(g[Y1], g["pred_y1_lgbm"]),

            "Y2_RMSE_lag1": _rmse(g[Y2], g["pred_y2_lag1"]),
            "Y2_RMSE_roll4": _rmse(g[Y2], g["pred_y2_roll4"]),
            "Y2_RMSE_lgbm": _rmse(g[Y2], g["pred_y2_lgbm"]),
            "Y2_MAE_lgbm": _mae(g[Y2], g["pred_y2_lgbm"]),
        })

    group_errors = (
        pd.DataFrame(rows)
        .sort_values(["treatment", "Y1_RMSE_lgbm"], ascending=[True, False])
        .reset_index(drop=True)
    )

    print("\n=== Group-level errors (RMSE/MAE) ===")
    print(group_errors.to_string(index=False))



FOCUS treatments: ['15元1張', '20元1張', '30元1張']
Rows: 72

=== Row-level results (truth + lag1 + roll4 + lgbm) ===
experiment_date treatment source ops_type_merged city_group  trip_cnt_per_user  pred_y1_lag1  pred_y1_roll4  pred_y1_lgbm  nonrepeat_cnt_per_user  pred_y2_lag1  pred_y2_roll4  pred_y2_lgbm
     2025-12-29     15元1張    隨機組    14天在其他尖峰預估車資         中區           0.192661      0.158273       0.153196      0.167503                0.256881      0.215827       0.205702      0.217252
     2025-12-29     15元1張    隨機組    14天在其他尖峰預估車資         北區           0.169591      0.214008       0.169938      0.171918                0.270955      0.322957       0.251182      0.293102
     2025-12-29     15元1張    隨機組    14天在其他尖峰預估車資         南區           0.128000      0.141304       0.120134      0.121114                0.168000      0.228261       0.185517      0.161229
     2025-12-29     15元1張    隨機組     14天在晚尖峰預估車資         中區           0.071429      0.488372       0.335436      0.153310          

C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\3581049564.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(grp_cols, dropna=False)
C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\3581049564.py:67: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for key, g in focus_df.groupby(grp_cols):


In [4]:
# ==========================
# Who is best per group? (lag1 vs roll4 vs lgbm)
# - Print count of groups each method wins
# - Bullet list groups + numbers
# ==========================

GROUP_KEY_COLS = ["treatment", "source", "ops_type_merged", "city_group"]

def _rmse(a, b):
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    return float(np.sqrt(np.mean((a - b) ** 2)))

def _mae(a, b):
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    return float(np.mean(np.abs(a - b)))

def best_model_by_group(df, y_col, pred_cols, group_cols, metric="RMSE"):
    """
    pred_cols: dict model_name -> pred_col_name
    metric: "RMSE" or "MAE" used to decide winner
    """
    rows = []
    for key, g in df.groupby(group_cols):
        y = g[y_col].astype(float).values
        out = {col: val for col, val in zip(group_cols, key)}
        out["target"] = y_col
        out["n"] = len(g)
        # means (helpful context)
        out["y_true_mean"] = float(np.mean(y))

        # errors + pred means
        errs = {}
        for m, pcol in pred_cols.items():
            p = g[pcol].astype(float).values
            out[f"{m}_pred_mean"] = float(np.mean(p))
            out[f"{m}_RMSE"] = _rmse(y, p)
            out[f"{m}_MAE"]  = _mae(y, p)
            errs[m] = out[f"{m}_{metric}"]

        # winner (smallest error)
        min_err = min(errs.values())
        winners = [m for m, e in errs.items() if np.isclose(e, min_err)]
        out["best_metric"] = metric
        out["best_error"] = float(min_err)
        out["best_model"] = winners[0] if len(winners) == 1 else "TIE:" + ",".join(sorted(winners))
        rows.append(out)

    res = pd.DataFrame(rows)
    # make it easier to read: sort by winner then best_error
    return res.sort_values(["target", "best_model", "best_error"]).reset_index(drop=True)

def print_winner_summary(res_df, pred_cols, title):
    print("\n" + "="*80)
    print(title)
    print("="*80)

    # 1) counts of winners
    counts = res_df["best_model"].value_counts().rename_axis("best_model").reset_index(name="n_groups")
    counts["pct"] = (counts["n_groups"] / counts["n_groups"].sum() * 100).round(2)
    print("\n[Winner counts]")
    print(counts.to_string(index=False))

    # 2) bullet list for each winner
    # (include ties as its own category if exists)
    for winner in counts["best_model"].tolist():
        sub = res_df[res_df["best_model"] == winner].copy()
        print("\n" + "-"*80)
        print(f"[{winner}] groups = {len(sub)}")
        print("-"*80)

        # bullet list: group key + n + true mean + each model error + each model pred mean
        for _, r in sub.iterrows():
            key_str = " | ".join([f"{c}={r[c]}" for c in GROUP_KEY_COLS])
            # errors
            err_str = ", ".join([
                f"{m}: {r[m+'_RMSE']:.6f} (MAE {r[m+'_MAE']:.6f})"
                for m in pred_cols.keys()
            ])
            # pred means
            pred_mean_str = ", ".join([
                f"{m}_pred_mean={r[m+'_pred_mean']:.6f}"
                for m in pred_cols.keys()
            ])
            print(f"• {key_str} | n={int(r['n'])} | y_true_mean={r['y_true_mean']:.6f}")
            print(f"  - RMSE/MAE: {err_str}")
            print(f"  - Pred means: {pred_mean_str}")

# ---- build maps (same as your earlier code) ----
pred_map_y1 = {
    "lag1": "pred_y1_lag1",
    "roll4": "pred_y1_roll4",
    "lgbm": "pred_y1_lgbm",
}
pred_map_y2 = {
    "lag1": "pred_y2_lag1",
    "roll4": "pred_y2_roll4",
    "lgbm": "pred_y2_lgbm",
}

# ---- compute per-group winners (use RMSE to decide best) ----
win_y1 = best_model_by_group(test, Y1, pred_map_y1, GROUP_KEY_COLS, metric="RMSE")
win_y2 = best_model_by_group(test, Y2, pred_map_y2, GROUP_KEY_COLS, metric="RMSE")

# ---- print results ----
print_winner_summary(win_y1, pred_map_y1, f"BEST model by group (target={Y1}, criterion=RMSE)")
print_winner_summary(win_y2, pred_map_y2, f"BEST model by group (target={Y2}, criterion=RMSE)")



BEST model by group (target=trip_cnt_per_user, criterion=RMSE)

[Winner counts]
best_model  n_groups   pct
      lgbm        73 38.02
     roll4        62 32.29
      lag1        57 29.69

--------------------------------------------------------------------------------
[lgbm] groups = 73
--------------------------------------------------------------------------------
• treatment=15元1張 | source=隨機組 | ops_type_merged=90天在尖峰預估車資 | city_group=北區 | n=1 | y_true_mean=0.062812
  - RMSE/MAE: lag1: 0.000729 (MAE 0.000729), roll4: 0.007579 (MAE 0.007579), lgbm: 0.000072 (MAE 0.000072)
  - Pred means: lag1_pred_mean=0.062082, roll4_pred_mean=0.055232, lgbm_pred_mean=0.062884
• treatment=15x2元1張 | source=隨機組 | ops_type_merged=喚回-高優惠敏感 | city_group=南區 | n=1 | y_true_mean=0.062500
  - RMSE/MAE: lag1: 0.010227 (MAE 0.010227), roll4: 0.053498 (MAE 0.053498), lgbm: 0.000318 (MAE 0.000318)
  - Pred means: lag1_pred_mean=0.072727, roll4_pred_mean=0.115998, lgbm_pred_mean=0.062818
• treatment=15元1張 | sou

C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\194011999.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for key, g in df.groupby(group_cols):
C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\194011999.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for key, g in df.groupby(group_cols):


In [5]:
# ==========================
# Backtest across weeks: who is best per group?
# (lag1 vs roll4 vs lgbm)
# ==========================

GROUP_KEY_COLS = ["treatment", "source", "ops_type_merged", "city_group"]

def _rmse(a, b):
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    return float(np.sqrt(np.mean((a - b) ** 2)))

def _mae(a, b):
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    return float(np.mean(np.abs(a - b)))

def train_and_predict_for_week(df, week_dt, feature_cols, cat_features, y_col, params):
    """Train on < week_dt, predict on == week_dt. Return pred array or None if empty."""
    tr = df[df[DATE_COL] < week_dt]
    te = df[df[DATE_COL] == week_dt]
    if tr.empty or te.empty:
        return None, None

    X_tr = tr[feature_cols].copy()
    y_tr = tr[y_col].astype(float).values
    X_te = te[feature_cols].copy()

    tr_df, va_df = make_time_val_split(tr, DATE_COL, n_val_weeks=4)
    X_tr2 = tr_df[feature_cols].copy()
    y_tr2 = tr_df[y_col].astype(float).values

    fit_kwargs = dict(categorical_feature=cat_features)
    m = lgb.LGBMRegressor(**params)

    if va_df is not None and not va_df.empty:
        X_va = va_df[feature_cols].copy()
        y_va = va_df[y_col].astype(float).values
        m.fit(
            X_tr2, y_tr2,
            eval_set=[(X_va, y_va)],
            eval_metric="l2",
            callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)],
            **fit_kwargs
        )
    else:
        m.fit(X_tr, y_tr, **fit_kwargs)

    pred = m.predict(X_te)
    return te, pred

# ---- feature columns (same as your original) ----
exclude = {Y1, Y2} | set(DROP_FEATURES)
feature_cols = [c for c in df.columns if c not in exclude]
if DATE_COL in feature_cols:
    feature_cols.remove(DATE_COL)
cat_features = [c for c in CAT_COLS if c in feature_cols]

# ---- model params (same as your original) ----
base_params = dict(
    objective="regression",
    n_estimators=8000,
    learning_rate=0.03,
    random_state=RANDOM_STATE,
    force_row_wise=True,
)

params_y1 = dict(
    num_leaves=31, max_depth=6, min_child_samples=300,
    subsample=0.8, colsample_bytree=0.8, bagging_freq=1,
    reg_alpha=0.0, reg_lambda=2.0, min_split_gain=0.0,
)
params_y2 = dict(
    num_leaves=31, max_depth=6, min_child_samples=200,
    subsample=0.8, colsample_bytree=0.9, bagging_freq=1,
    reg_alpha=0.0, reg_lambda=2.0, min_split_gain=0.0,
)

params_full_y1 = {**base_params, **params_y1}
params_full_y2 = {**base_params, **params_y2}

# ---- choose weeks to backtest (exclude the very first weeks if lag/roll may be NaN) ----
all_weeks = sorted(df[DATE_COL].dt.date.unique())
all_weeks = [pd.to_datetime(d) for d in all_weeks]

# 可自行調整：至少要有足夠前期資料才訓練（例如跳過最前面 6 週）
SKIP_FIRST_WEEKS = 6
weeks_to_eval = all_weeks[SKIP_FIRST_WEEKS:]

print("Weeks to eval:", len(weeks_to_eval), "from", weeks_to_eval[0].date(), "to", weeks_to_eval[-1].date())

# ---- collect per-week predictions ----
records = []

for w in weeks_to_eval:
    # Y1
    te1, pred1 = train_and_predict_for_week(df, w, feature_cols, cat_features, Y1, params_full_y1)
    if te1 is not None:
        tmp = te1[GROUP_KEY_COLS + [DATE_COL, Y1, Y1_LAG1, Y1_ROLL4]].copy()
        tmp["target"] = Y1
        tmp["pred_lag1"] = tmp[Y1_LAG1].astype(float)
        tmp["pred_roll4"] = tmp[Y1_ROLL4].astype(float)
        tmp["pred_lgbm"] = pred1.astype(float)
        tmp = tmp.rename(columns={Y1: "y_true"})
        records.append(tmp[GROUP_KEY_COLS + [DATE_COL, "target", "y_true", "pred_lag1", "pred_roll4", "pred_lgbm"]])

    # Y2
    te2, pred2 = train_and_predict_for_week(df, w, feature_cols, cat_features, Y2, params_full_y2)
    if te2 is not None:
        tmp = te2[GROUP_KEY_COLS + [DATE_COL, Y2, Y2_LAG1, Y2_ROLL4]].copy()
        tmp["target"] = Y2
        tmp["pred_lag1"] = tmp[Y2_LAG1].astype(float)
        tmp["pred_roll4"] = tmp[Y2_ROLL4].astype(float)
        tmp["pred_lgbm"] = pred2.astype(float)
        tmp = tmp.rename(columns={Y2: "y_true"})
        records.append(tmp[GROUP_KEY_COLS + [DATE_COL, "target", "y_true", "pred_lag1", "pred_roll4", "pred_lgbm"]])

pred_long = pd.concat(records, ignore_index=True)
print("Collected rows:", len(pred_long))

# ---- compute winner per group, per target across weeks ----
def winner_table(pred_long, group_cols, metric="RMSE", min_n=3):
    rows = []
    for key, g in pred_long.groupby(group_cols + ["target"]):
        y = g["y_true"].astype(float).values
        n = len(g)

        # require at least min_n weeks to be meaningful
        if n < min_n:
            continue

        e = {}
        e["lag1_RMSE"] = _rmse(y, g["pred_lag1"].values)
        e["roll4_RMSE"] = _rmse(y, g["pred_roll4"].values)
        e["lgbm_RMSE"] = _rmse(y, g["pred_lgbm"].values)

        e["lag1_MAE"] = _mae(y, g["pred_lag1"].values)
        e["roll4_MAE"] = _mae(y, g["pred_roll4"].values)
        e["lgbm_MAE"] = _mae(y, g["pred_lgbm"].values)

        if metric == "RMSE":
            pick = {"lag1": e["lag1_RMSE"], "roll4": e["roll4_RMSE"], "lgbm": e["lgbm_RMSE"]}
        else:
            pick = {"lag1": e["lag1_MAE"], "roll4": e["roll4_MAE"], "lgbm": e["lgbm_MAE"]}

        min_err = min(pick.values())
        winners = [m for m, val in pick.items() if np.isclose(val, min_err)]
        best_model = winners[0] if len(winners) == 1 else "TIE:" + ",".join(sorted(winners))

        out = {col: val for col, val in zip(group_cols + ["target"], key)}
        out["n_weeks"] = n
        out["y_true_mean"] = float(np.mean(y))
        out.update(e)
        out["best_metric"] = metric
        out["best_error"] = float(min_err)
        out["best_model"] = best_model
        rows.append(out)

    res = pd.DataFrame(rows)
    return res.sort_values(["target", "best_model", "best_error"]).reset_index(drop=True)

win = winner_table(pred_long, GROUP_KEY_COLS, metric="RMSE", min_n=3)

# ---- print counts + bullet list ----
def print_winner(win_df, target):
    sub = win_df[win_df["target"] == target].copy()
    print("\n" + "="*90)
    print(f"Winner by group across weeks | target={target} | criterion=RMSE | groups={len(sub)}")
    print("="*90)

    counts = sub["best_model"].value_counts().rename_axis("best_model").reset_index(name="n_groups")
    counts["pct"] = (counts["n_groups"] / counts["n_groups"].sum() * 100).round(2)
    print("\n[Winner counts]")
    print(counts.to_string(index=False))

    for winner in counts["best_model"].tolist():
        gdf = sub[sub["best_model"] == winner].copy()
        print("\n" + "-"*90)
        print(f"[{winner}] groups={len(gdf)}")
        print("-"*90)
        for _, r in gdf.iterrows():
            key_str = " | ".join([f"{c}={r[c]}" for c in GROUP_KEY_COLS])
            print(f"• {key_str} | n_weeks={int(r['n_weeks'])} | y_true_mean={r['y_true_mean']:.6f}")
            print(f"  - RMSE: lag1={r['lag1_RMSE']:.6f}, roll4={r['roll4_RMSE']:.6f}, lgbm={r['lgbm_RMSE']:.6f}")
            print(f"  - MAE : lag1={r['lag1_MAE']:.6f}, roll4={r['roll4_MAE']:.6f}, lgbm={r['lgbm_MAE']:.6f}")

print_winner(win, Y1)
print_winner(win, Y2)


Weeks to eval: 8 from 2025-11-17 to 2026-01-05
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Total Bins 22
[LightGBM] [Info] Number of data points in the train set: 576, number of used features: 3
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Start training from score 0.106581
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_23464\1230754957.py:122: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for key, g in pred_long.groupby(group_cols + ["target"]):
